In [1]:
# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from jupyter_dash import JupyterDash  

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

# author SNHU  cs program
# dashboard to display anmial rescue inofmration from AAC Mongo DB

# updated 10-19-24 Nicholas Rusinski
# added DB conection
# added filtering options to data table
# added data views bsed on prefered rescue breeds
# added dynamic pie chart to help explain filtred data table


# Hard coded username and password to connect to DB
username = "aacuser"
password = "SNHU092724"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Class read method must support return of list object and accept projection json input
# Sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read())

# MongoDB v5+ is going to return the '_id' column and that is going to have an
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

# Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

# Dashboard Layout / View
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Dashboard layout
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),

    # Logo with link to SNHU home page link
    html.Div(
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
            href='https://www.snhu.edu',  
            target='_blank'  
        ),
        style={'textAlign': 'center'}
    ),

    # Name for identifier
    html.Div(
        html.H3('Designed and built by Nicholas Rusinski'),
        style={'textAlign': 'center'}
    ),

    # Not sure what this does; consider removing
    html.Div(id='hidden-div', style={'display': 'none'}),

    # Drop down menu with filtering options
    html.Div([
        html.Label('Select Rescue Type:'),
        dcc.Dropdown(
            id='rescue-type-dropdown',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'disaster'},
                {'label': 'All animals', 'value': 'reset'}
            ],
            value='reset',  
            clearable=False  
        ),
    ], style={'width': '50%', 'margin': 'auto'}),  

    # Horizontal rule
    html.Hr(),
    
    # data table starts here
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),

        # adds filtering sorting and pagiantion to dashboard
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=50,
        style_table={'overflowX': 'auto'},
        fixed_rows={'headers': True},
        style_cell={
            'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis'
        }
    ),

    html.Br(),
    html.Hr(),

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# call back to get input from html drop down field
@app.callback(Output('datatable-id', 'data'),
              Output('datatable-id', 'columns'),
              [Input('rescue-type-dropdown', 'value')])
# updates table based on drop down result
def update_dashboard(filter_type):
    
    if filter_type == 'water':
        query = {
            "breed": {
                "$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26,
                "$lte": 156
            }
        }
        
    elif filter_type == 'mountain':
        query = {
            "breed": {
                "$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26,
                "$lte": 156
            }
        }

    elif filter_type == 'disaster':
        query = {
            "breed": {
                "$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20,
                "$lte": 300
            }
        }
    
    # this gets called at start to populate results on page navigation
    else:  
        query = None
    
    # Class read method must support return of list object and accept projection json input
    # Sending the read method an empty document requests all documents be returned
    df = pd.DataFrame.from_records(db.read(query))
    # MongoDB v5+ is going to return the '_id' column and that is going to have an
    # invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
    # it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
    # inplace=True - it will return a new dataframe that does not contain the dropped column(s)
    df.drop(columns=['_id'], inplace=True)
    
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')

    return data, columns

#call back to send pie chart data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('rescue-type-dropdown', 'value')])
# updates pie chart with new data
def update_graphs(viewData, filter_type):
    if viewData is None:
        return
    
    df = pd.DataFrame.from_dict(viewData)
    
    # filters out breeds that take up less then 1% or chart these are popualted as "misc" breeds
    breed_counts = df['breed'].value_counts()
    total_counts = breed_counts.sum()
    breed_percentages = (breed_counts / total_counts) * 100
    
    chart_df = breed_percentages.reset_index()
    chart_df.columns = ['breed', 'percentage']
    
    chart_df['breed'] = chart_df.apply(
        lambda row: 'Misc breeds' if row['percentage'] < 1 else row['breed'], axis=1
    )
    
    final_df = chart_df.groupby('breed', as_index=False).sum()
    
    if filter_type == 'reset':
        title = 'Preferred Animals Breeds'
    else:
        title = f'Preferred {filter_type.capitalize()} Rescue Animal Breeds'
    
    return [
        dcc.Graph(
            figure=px.pie(final_df, names='breed', values='percentage', title=title)
        )
    ]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    if index is None:
        row = 0
    else:
        row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:26539/
